In [1]:
import pandas as pd 
import numpy as np
import sys

In [2]:
# df = pd.read_csv('data_sources/individual_cases_ontario.csv')
df = pd.read_csv('dimensions/data_subset.csv')

In [3]:
df.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,6323,2021-01-29,2021-02-01,2021-02-01,2020-12-24,<20,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
1,6329,2020-03-11,2020-12-05,2020-12-05,2020-09-29,20s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,7559,2020-03-23,2020-12-12,2020-12-12,2020-12-11,30s,FEMALE,CC,Resolved,NaN,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
3,7888,2020-04-29,2020-10-06,2020-10-06,2020-08-21,50s,MALE,NO KNOWN EPI LINK,Resolved,NaN,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
4,26512,2021-01-26,2021-02-06,2021-02-06,2020-09-24,80s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [4]:
#Drop PHU related columns (already staged in PHU dimension)
df.drop(columns=['Reporting_PHU_ID', 'Reporting_PHU_Address', 'Reporting_PHU_Postal_Code', 'Reporting_PHU_Website',
        'Reporting_PHU_Latitude', 'Reporting_PHU_Longitude'], inplace=True)

In [5]:
df['Outbreak_Related'].fillna('No', inplace=True)
df.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU,Reporting_PHU_City
0,6323,2021-01-29,2021-02-01,2021-02-01,2020-12-24,<20,MALE,CC,Resolved,No,Peel Public Health,Mississauga
1,6329,2020-03-11,2020-12-05,2020-12-05,2020-09-29,20s,FEMALE,CC,Resolved,No,Peel Public Health,Mississauga
2,7559,2020-03-23,2020-12-12,2020-12-12,2020-12-11,30s,FEMALE,CC,Resolved,No,York Region Public Health Services,Newmarket
3,7888,2020-04-29,2020-10-06,2020-10-06,2020-08-21,50s,MALE,NO KNOWN EPI LINK,Resolved,No,York Region Public Health Services,Newmarket
4,26512,2021-01-26,2021-02-06,2021-02-06,2020-09-24,80s,FEMALE,OB,Resolved,Yes,Toronto Public Health,Toronto


In [6]:
len(df)

161219

In [7]:
df.columns = ['row_id', 'onset_date', 'reported_date', 'test_date', 'specimen_date', 
             'age_group', 'client_gender', 'case_acquisition_info', 'outcome1',
             'outbreak_related', 'phu', 'phu_city']

In [8]:
df.head()

,row_id,onset_date,reported_date,test_date,specimen_date,age_group,client_gender,case_acquisition_info,outcome1,outbreak_related,phu,phu_city
0,6323,2021-01-29,2021-02-01,2021-02-01,2020-12-24,<20,MALE,CC,Resolved,No,Peel Public Health,Mississauga
1,6329,2020-03-11,2020-12-05,2020-12-05,2020-09-29,20s,FEMALE,CC,Resolved,No,Peel Public Health,Mississauga
2,7559,2020-03-23,2020-12-12,2020-12-12,2020-12-11,30s,FEMALE,CC,Resolved,No,York Region Public Health Services,Newmarket
3,7888,2020-04-29,2020-10-06,2020-10-06,2020-08-21,50s,MALE,NO KNOWN EPI LINK,Resolved,No,York Region Public Health Services,Newmarket
4,26512,2021-01-26,2021-02-06,2021-02-06,2020-09-24,80s,FEMALE,OB,Resolved,Yes,Toronto Public Health,Toronto


In [9]:
len(df)

161219

In [10]:
# Check for nulls in the dates 
dates = ['onset_date', 'reported_date', 'specimen_date', 'test_date']
nan_found = False
for date in dates:
    if df[date].isnull().values.any():
        nan_found = True
        print(f"NaN found in {date} col")

if not nan_found:
    print('Date columns are valid')

NaN found in specimen_date col
NaN found in test_date col


In [11]:
nan_speci_rows = df['specimen_date'].isnull()
df.loc[nan_speci_rows]

,row_id,onset_date,reported_date,test_date,specimen_date,age_group,client_gender,case_acquisition_info,outcome1,outbreak_related,phu,phu_city
1555,30395,2020-10-13,2020-10-13,NaN,NaN,50s,MALE,CC,Resolved,No,York Region Public Health Services,Newmarket
2626,31853,2020-10-12,2020-10-16,2020-10-16,NaN,50s,FEMALE,NO KNOWN EPI LINK,Resolved,No,Durham Region Health Department,Whitby
2663,31899,2020-10-17,2020-10-17,NaN,NaN,<20,MALE,CC,Resolved,Yes,York Region Public Health Services,Newmarket
4240,34005,2020-09-25,2020-10-20,NaN,NaN,50s,FEMALE,CC,Resolved,No,York Region Public Health Services,Newmarket
4241,34006,2020-10-20,2020-10-20,NaN,NaN,20s,MALE,NO KNOWN EPI LINK,Resolved,No,York Region Public Health Services,Newmarket
...,...,...,...,...,...,...,...,...,...,...,...,...
155938,283852,2020-12-05,2020-12-21,NaN,NaN,60s,FEMALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto
157463,286671,2020-12-26,2020-12-30,NaN,NaN,20s,MALE,CC,Resolved,No,Ottawa Public Health,Ottawa
159918,290938,2020-12-21,2020-12-23,NaN,NaN,30s,MALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto
160285,291517,2020-12-19,2020-12-24,NaN,NaN,50s,FEMALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto


In [12]:
# Check how many rows 
len(df.specimen_date[nan_speci_rows])

196

In [13]:
nan_test_rows = df['test_date'].isnull()
df.loc[nan_test_rows]

,row_id,onset_date,reported_date,test_date,specimen_date,age_group,client_gender,case_acquisition_info,outcome1,outbreak_related,phu,phu_city
160,28353,2020-09-27,2020-10-06,NaN,2020-09-29,30s,MALE,CC,Resolved,No,York Region Public Health Services,Newmarket
432,28781,2020-10-06,2020-10-07,NaN,2020-10-06,20s,MALE,CC,Resolved,No,Halton Region Health Department,Oakville
435,28784,2020-09-29,2020-10-07,NaN,2020-10-04,20s,MALE,CC,Resolved,No,Halton Region Health Department,Oakville
704,29168,2020-09-29,2020-10-08,NaN,2020-10-03,40s,UNSPECIFIED,CC,Resolved,No,York Region Public Health Services,Newmarket
1555,30395,2020-10-13,2020-10-13,NaN,NaN,50s,MALE,CC,Resolved,No,York Region Public Health Services,Newmarket
...,...,...,...,...,...,...,...,...,...,...,...,...
161074,292848,2020-12-12,2020-12-15,NaN,2020-12-12,60s,MALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto
161076,292850,2020-12-14,2020-12-18,NaN,2020-12-14,40s,MALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto
161104,292892,2021-01-12,2021-01-15,NaN,2021-01-12,30s,FEMALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto
161105,292893,2021-01-04,2021-01-15,NaN,2021-01-09,20s,FEMALE,NO KNOWN EPI LINK,Resolved,No,Toronto Public Health,Toronto


In [14]:
# Check how many rows 
len(df.test_date[nan_test_rows])

7874

In [15]:
def get_precentage_missing():
    den = len(df)
    spec_num = df['specimen_date'].isnull().sum()
    test_num = df['test_date'].isnull().sum()
    return (round(spec_num/den,4)*100, round(test_num/den,4)*100)

spec_perc, test_perc = get_precentage_missing()

print(spec_perc, test_perc)

0.12 4.88


In [16]:
# Drop rows with specimen date / test date have null value 
df.dropna(subset=['specimen_date', 'test_date'], inplace=True)

# Check for nulls in the dates 
dates = ['onset_date', 'reported_date', 'specimen_date', 'test_date']
nan_found = False
for date in dates:
    if df[date].isnull().values.any():
        nan_found = True
        print(f"NaN found in {date} col")

if not nan_found:
    print('Date columns are valid')

Date columns are valid


In [17]:
# Convert date columns to dates 
df['onset_date'] = pd.to_datetime(df['onset_date'])
df['reported_date'] = pd.to_datetime(df['reported_date'])
df['test_date'] = pd.to_datetime(df['test_date'])
df['specimen_date'] = pd.to_datetime(df['specimen_date'])

In [18]:
def date_bounds(date):
    print(f"{date}: {df[date].min()} - {df[date].max()}")
    
dates = ['onset_date','reported_date','test_date', 'specimen_date'] 
for d in dates:
    date_bounds(d)

onset_date: 2020-01-05 00:00:00 - 2021-02-06 00:00:00
reported_date: 2020-10-06 00:00:00 - 2021-02-06 00:00:00
test_date: 2020-01-03 00:00:00 - 2021-02-19 00:00:00
specimen_date: 2020-01-09 00:00:00 - 2021-02-18 00:00:00


In [19]:
# Load the dimensions to map 
patient_dimension = pd.read_csv('dimensions/patient_dimension.csv')
phu_dimension = pd.read_csv('dimensions/PHU_location_dimension.csv')
date_dimension = pd.read_csv('dimensions/date_dimension.csv')
special_measures_dimension = pd.read_csv('dimensions/special_measures_dimension.csv')

In [20]:
# Create fact table
fact_table = pd.DataFrame(columns=['Onset_date_ID','Reported_date_ID', 'Test_date_ID', 'Specimen_date_ID', 'Patient_ID',
                                  'PHU_Location_ID', 'Mobility_ID', 'Special_Measures_ID', 'Weather_ID', 'outcome'])

In [21]:
# Convert dates to dates 
date_dimension['date'] = pd.to_datetime(date_dimension['date'])
special_measures_dimension['start_date'] = pd.to_datetime(special_measures_dimension['start_date'])
special_measures_dimension['end_date'] = pd.to_datetime(special_measures_dimension['end_date'])

In [22]:
# Check all possible values 
set(df['outcome1'])

{'Fatal', 'Not Resolved', 'Resolved'}

In [23]:
# Check rows with valid date orders 
df[
    ~((df.onset_date <= df.specimen_date) &
    (df.specimen_date <= df.test_date) & 
    (df.test_date <= df.reported_date))
]

,row_id,onset_date,reported_date,test_date,specimen_date,age_group,client_gender,case_acquisition_info,outcome1,outbreak_related,phu,phu_city
0,6323,2021-01-29,2021-02-01,2021-02-01,2020-12-24,<20,MALE,CC,Resolved,No,Peel Public Health,Mississauga
4,26512,2021-01-26,2021-02-06,2021-02-06,2020-09-24,80s,FEMALE,OB,Resolved,Yes,Toronto Public Health,Toronto
5,26850,2020-10-05,2020-10-09,2020-10-09,2020-10-01,40s,FEMALE,OB,Resolved,Yes,Durham Region Health Department,Whitby
26,28091,2020-12-06,2020-12-09,2020-12-09,2020-09-30,30s,FEMALE,CC,Resolved,No,Durham Region Health Department,Whitby
221,28428,2020-09-27,2020-10-07,2020-10-08,2020-09-27,<20,MALE,NO KNOWN EPI LINK,Resolved,No,Peel Public Health,Mississauga
...,...,...,...,...,...,...,...,...,...,...,...,...
161155,292984,2020-12-19,2020-12-20,2020-12-20,2020-10-30,90+,MALE,OB,Fatal,Yes,Toronto Public Health,Toronto
161170,293019,2021-01-05,2021-01-08,2021-01-08,2020-12-01,50s,MALE,OB,Resolved,Yes,Toronto Public Health,Toronto
161178,293028,2020-11-20,2020-11-21,2020-11-21,2020-11-17,30s,FEMALE,MISSING INFORMATION,Resolved,No,Toronto Public Health,Toronto
161189,293045,2021-01-10,2021-01-11,2021-01-11,2020-11-29,40s,FEMALE,MISSING INFORMATION,Resolved,Yes,Toronto Public Health,Toronto


In [24]:
def get_precentage_invalid_dates():
    den = len(df)
    invalid_num = len(df[ ~((df.onset_date <= df.specimen_date) & (df.specimen_date <= df.test_date) & 
    (df.test_date <= df.reported_date))])
    return round(invalid_num/den,4)*100

get_precentage_invalid_dates()

3.42

In [25]:
# Drop rows with invalid timelines 
df = df[
    ((df.onset_date <= df.specimen_date) &
    (df.specimen_date <= df.test_date) & 
    (df.test_date <= df.reported_date))
]

In [26]:
df.head()

,row_id,onset_date,reported_date,test_date,specimen_date,age_group,client_gender,case_acquisition_info,outcome1,outbreak_related,phu,phu_city
1,6329,2020-03-11,2020-12-05,2020-12-05,2020-09-29,20s,FEMALE,CC,Resolved,No,Peel Public Health,Mississauga
2,7559,2020-03-23,2020-12-12,2020-12-12,2020-12-11,30s,FEMALE,CC,Resolved,No,York Region Public Health Services,Newmarket
3,7888,2020-04-29,2020-10-06,2020-10-06,2020-08-21,50s,MALE,NO KNOWN EPI LINK,Resolved,No,York Region Public Health Services,Newmarket
6,27494,2020-10-02,2020-10-06,2020-10-06,2020-10-02,20s,MALE,CC,Resolved,No,Durham Region Health Department,Whitby
7,27535,2020-09-26,2020-10-06,2020-10-06,2020-09-30,<20,FEMALE,TRAVEL,Resolved,No,York Region Public Health Services,Newmarket


In [27]:
patient_dimension.head()

,patient_key,row_id,acquisition_group,outbreak_related,gender,age_group
0,0,6323,CC,No,MALE,<20
1,1,6329,CC,No,FEMALE,20s
2,2,7559,CC,No,FEMALE,30s
3,3,7888,NO KNOWN EPI LINK,No,MALE,50s
4,4,26512,OB,Yes,FEMALE,80s


In [28]:
def is_nearest(candidate, items, pivot):
    return candidate == nearest(items, pivot)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

y = 161219
for idx, row in df.iterrows():
    sys.stdout.write("\r{0} out of {1} generated...".format(idx,y))
    date_vals = [
        date_dimension[date_dimension["date"]==row["onset_date"]]["date_id"].values[0],
        date_dimension[date_dimension["date"]==row["reported_date"]]["date_id"].values[0],
        date_dimension[date_dimension["date"]==row["test_date"]]["date_id"].values[0],
        date_dimension[date_dimension["date"]==row["specimen_date"]]["date_id"].values[0],
    ]
    
    patient_id = [
        patient_dimension[patient_dimension["row_id"]==row["row_id"]]['patient_key'].values[0]
    ]
    
    phu_id = [
        phu_dimension[phu_dimension['PHU_name']==row['phu']]['PHU_id'].values[0]
    ]
    
    mobility_id = [-1]
    # Find measures with nearest start date to the row date
    nearest_start_date = pd.to_datetime(nearest(special_measures_dimension.start_date, row['reported_date']))

    candidate_measures = special_measures_dimension.loc[
        (special_measures_dimension.start_date == nearest_start_date)
    ]
    # Select the measure with the highest value (the stricter)
    special_measures_id = [candidate_measures['value'].idxmax()]

    
    weather_id = [-1]
    
    outcome_measure = [row['outcome1']]
    
    fact_row = date_vals + patient_id + phu_id + mobility_id + special_measures_id + weather_id + outcome_measure
    
    fact_table.loc[len(fact_table)] = fact_row

161218 out of 161219 generated...

In [29]:
fact_table.head()

,Onset_date_ID,Reported_date_ID,Test_date_ID,Specimen_date_ID,Patient_ID,PHU_Location_ID,Mobility_ID,Special_Measures_ID,Weather_ID,outcome
0,20200311,20201205,20201205,20200929,1,2253,-1,5,-1,Resolved
1,20200323,20201212,20201212,20201211,2,2270,-1,3,-1,Resolved
2,20200429,20201006,20201006,20200821,3,2270,-1,14,-1,Resolved
3,20201002,20201006,20201006,20201002,6,2230,-1,14,-1,Resolved
4,20200926,20201006,20201006,20200930,7,2270,-1,14,-1,Resolved


In [30]:
fact_table.to_csv("dimensions/fact_table_init.csv", index=False)

In [31]:
fact_table = pd.read_csv('dimensions/fact_table_init.csv')

In [32]:
fact_table_m = pd.DataFrame(columns=['Onset_date_ID','Reported_date_ID', 'Test_date_ID', 'Specimen_date_ID', 'Patient_ID',
                                  'PHU_Location_ID', 'Mobility_ID', 'Special_Measures_ID', 'Weather_ID', 'resolved', 
                                   'unresolved', 'fatal'])

In [33]:
len(fact_table)

148053

In [35]:
for idx, row in fact_table.iterrows():
    sys.stdout.write("\r{0} out of {1} generated...".format(idx,y))

    fact_row = []
    fact_row.append(row['Onset_date_ID']) #Onset date
    fact_row.append(row['Reported_date_ID']) #reported date
    fact_row.append(row['Test_date_ID']) #test date
    fact_row.append(row['Specimen_date_ID']) #specimen date
    fact_row.append(row['Patient_ID']) #patient id 
    fact_row.append(row['PHU_Location_ID']) #PHU location id
    fact_row.append(row['Mobility_ID']) #mobility id 
    fact_row.append(row['Special_Measures_ID']) #special measures id
    fact_row.append(row['Weather_ID']) #weather id 
    if row['outcome'] == 'Resolved':
        outcome_rows = ['yes', 'no', 'no']
    elif row['outcome'] == 'Not Resolved':
        outcome_rows = ['no', 'yes', 'no']
    elif row['outcome'] == 'Fatal':
        outcome_rows = ['no', 'no', 'yes']
    
    fact_row += outcome_rows 
    fact_table_m.loc[len(fact_table_m)] = fact_row
        

148052 out of 161219 generated...

In [36]:
len(fact_table_m)

148053

In [37]:
fact_table_m.tail()

,Onset_date_ID,Reported_date_ID,Test_date_ID,Specimen_date_ID,Patient_ID,PHU_Location_ID,Mobility_ID,Special_Measures_ID,Weather_ID,resolved,unresolved,fatal
148048,20201022,20201023,20201023,20201022,161213,3895,-1,1,-1,no,no,yes
148049,20201227,20201229,20201229,20201227,161214,3895,-1,12,-1,yes,no,no
148050,20201204,20201207,20201207,20201206,161215,3895,-1,3,-1,yes,no,no
148051,20201229,20210101,20210101,20201229,161217,3895,-1,12,-1,yes,no,no
148052,20201204,20201205,20201205,20201204,161218,3895,-1,5,-1,yes,no,no


In [38]:
fact_table_m.to_csv("dimensions/fact_table_m.csv", index=False)